In [ ]:
import colorsys
import random
import tkinter as tk

import pandas as pd

# THIS KEEPS THE COLOR OF THE CELLS CONSTANT AND CHANGES THE COLOR OF THE WINDOW use the other file for the opposite effect

# Cell class to represent each cell in the grid


class Cell:
    def __init__(self, value, schema='default_schema', machine_set='default_set'):
        self.value = value
        self.schema = schema
        self.machine_set = machine_set
        self.color = self.calculate_color()

    # Calculate the color based on the value, schema, and machine set
    def calculate_color(self):
        hue = (self.value * 36) % 360
        if self.schema == 'schema1' and self.machine_set == 'set1':
            hue += 50
        elif self.schema == 'schema2' and self.machine_set == 'set2':
            hue += 100
        r, g, b = colorsys.hls_to_rgb(hue / 360, 0.5, 1)
        return f'#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}'

# ResonanceGrid class to manage the grid


class ResonanceGrid:
    def __init__(self, matrix):
        self.grid = [[Cell(value) for value in row] for row in matrix]
        self.current_frequency = 1

    # Display the grid on a canvas
    def display(self, canvas):
        cell_size = 10
        for x, row in enumerate(self.grid):
            for y, cell in enumerate(row):
                canvas.create_rectangle(y * cell_size, x * cell_size,
                                        (y + 1) * cell_size, (x + 1) * cell_size,
                                        fill=cell.color, outline='')

    # Calculate the average color of the grid
    def average_color(self):
        total_r = total_g = total_b = 0
        total_cells = len(self.grid) * len(self.grid[0])
        for row in self.grid:
            for cell in row:
                r, g, b = int(cell.color[1:3], 16), int(
                    cell.color[3:5], 16), int(cell.color[5:7], 16)
                total_r += r
                total_g += g
                total_b += b
        avg_r = total_r / total_cells
        avg_g = total_g / total_cells
        avg_b = total_b / total_cells
        return f'#{int(avg_r):02x}{int(avg_g):02x}{int(avg_b):02x}'

    # Apply resonance effect to the grid
    def apply_resonance(self, frequency, root):
        self.current_frequency = frequency
        for row in self.grid:
            for cell in row:
                cell.value = (cell.value + frequency) % 10
                cell.color = cell.calculate_color()
        avg_color = self.average_color()
        root.title(f"Resonance Grid Simulation - Average Color: {avg_color}")

# Main function to set up the Tkinter window and grid


def main():
    root = tk.Tk()
    root.title("Resonance Grid Simulation")
    base_grid = pd.read_csv('a.csv', header=None).values.tolist()

    # Replace with your method of generating grid values
    resonance_grid = ResonanceGrid(base_grid)

    canvas = tk.Canvas(root, width=1000, height=1000)
    canvas.pack()

    # Redraw the grid
    def redraw():
        canvas.delete("all")
        resonance_grid.display(canvas)

    # Event handler for mouse clicks
    def on_click(event):
        frequency = random.randint(1, 9)
        resonance_grid.apply_resonance(frequency, root)
        redraw()

    canvas.bind("<Button-1>", on_click)
    redraw()
    root.mainloop()


if __name__ == "__main__":
    main()


In [ ]:
pip install -r requirements.txt

In [ ]:
import pandas as pd
import random


class Cell:
    def __init__(self, value):
        self.value = value
        self.color = 'white'  # Initially colorless

    def spin_effect(self, neighbors):
        # Define the whirlwind effect here
        # For simplicity, let's say the value becomes the average of its neighbors
        self.value = sum(neighbors) / len(neighbors)

    def update_color(self):
        # Update the color based on the new value
        # Here we can define a new color mapping based on the value
        if self.value < 2:
            self.color = 'blue'
        elif self.value < 3:
            self.color = 'green'
        else:
            self.color = 'red'


class GameGrid:
    def __init__(self, matrix):
        self.grid = [[Cell(value) for value in row] for row in matrix]

    def display(self):
        for row in self.grid:
            print(' '.join(cell.color[0].upper() for cell in row))

    def spin_cells(self):
        for x in range(len(self.grid)):
            for y in range(len(self.grid[0])):
                neighbors = self.get_neighbors_values(x, y)
                self.grid[x][y].spin_effect(neighbors)
        # After spinning, update colors
        for x in range(len(self.grid)):
            for y in range(len(self.grid[0])):
                self.grid[x][y].update_color()

    def get_neighbors_values(self, x, y):
        # Get the values of neighboring cells
        neighbors = []
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < len(self.grid) and 0 <= ny < len(self.grid[0]) and (dx, dy) != (0, 0):
                    neighbors.append(self.grid[nx][ny].value)
        return neighbors


def main():
    # Initialize the game grid with a sample matrix
    base_grid = pd.read_csv('a.csv', header=None).values.tolist()
    game_grid = GameGrid(base_grid)

    while True:
        game_grid.display()
        input("Press Enter to spin cells...")
        game_grid.spin_cells()


if __name__ == "__main__":
    main()


In [2]:
import tkinter as tk
import pandas as pd
import random

## This does a weird whirlwind effect on the cells and affects a random other cell

class Cell:
    def __init__(self, value, x, y):
        self.value = value
        self.x = x
        self.y = y
        self.color = self.calculate_color()

    def calculate_color(self):
        # Custom color based on position and value
        red = (self.value * 123 + self.x * 45) % 256
        green = (self.value * 156 + self.y * 67) % 256
        blue = (self.value * 189 + (self.x + self.y) * 89) % 256
        return f'#{int(red):02x}{int(green):02x}{int(blue):02x}'

    def update_value_based_on_spiral(self, spiral_power):
        self.value = (self.value + spiral_power) % 360
        self.color = self.calculate_color()


class GameGrid:
    def __init__(self, matrix):
        self.grid = [[Cell(value, x, y) for y, value in enumerate(row)]
                     for x, row in enumerate(matrix)]

    def apply_spiral_effect(self, x_center, y_center, spiral_power):
        for x, row in enumerate(self.grid):
            for y, cell in enumerate(row):
                distance = ((x - x_center) ** 2 + (y - y_center) ** 2) ** 0.5
                if distance <= spiral_power:
                    cell.update_value_based_on_spiral(spiral_power)

    def display(self, canvas):
        cell_size = 10  # Size of each cell in pixels
        for x, row in enumerate(self.grid):
            for y, cell in enumerate(row):
                canvas.create_rectangle(y * cell_size, x * cell_size,
                                        (y + 1) * cell_size, (x + 1) * cell_size,
                                        fill=cell.color, outline='')


def main():
    root = tk.Tk()
    root.title("Game Grid")

    base_grid = pd.read_csv('./a.csv', header=None).values.tolist()
    game_grid = GameGrid(base_grid)

    canvas = tk.Canvas(root, width=1000, height=1000)
    canvas.pack()

    def redraw():
        canvas.delete("all")
        game_grid.display(canvas)

    def on_click(event):
        x, y = event.x // 10, event.y // 10
        spiral_power = 5
        game_grid.apply_spiral_effect(x, y, spiral_power)
        redraw()

    canvas.bind("<Button-1>", on_click)
    redraw()
    root.mainloop()


if __name__ == "__main__":
    main()


In [1]:
import tkinter as tk
import random
import colorsys
import pandas as pd

# Cell class to represent each cell in the grid


class Cell:
    def __init__(self, value, schema='default_schema', machine_set='default_set'):
        self.value = value
        self.schema = schema
        self.machine_set = machine_set
        self.color = self.calculate_color()

    # Calculate the color based on the value, schema, and machine set
    def calculate_color(self):
        hue = (self.value * 36) % 360
        if self.schema == 'schema1' and self.machine_set == 'set1':
            hue += 50
        elif self.schema == 'schema2' and self.machine_set == 'set2':
            hue += 100
        r, g, b = colorsys.hls_to_rgb(hue / 360, 0.5, 1)
        return f'#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}'

# ResonanceGrid class to manage the grid


class ResonanceGrid:
    def __init__(self, matrix):
        self.grid = [[Cell(value) for value in row] for row in matrix]
        self.current_frequency = 1

    # Display the grid on a canvas
    def display(self, canvas):
        cell_size = 10
        for x, row in enumerate(self.grid):
            for y, cell in enumerate(row):
                canvas.create_rectangle(y * cell_size, x * cell_size,
                                        (y + 1) * cell_size, (x + 1) * cell_size,
                                        fill=cell.color, outline='')

    # Calculate the average color of the grid
    def average_color(self):
        total_r = total_g = total_b = 0
        total_cells = len(self.grid) * len(self.grid[0])
        for row in self.grid:
            for cell in row:
                r, g, b = int(cell.color[1:3], 16), int(
                    cell.color[3:5], 16), int(cell.color[5:7], 16)
                total_r += r
                total_g += g
                total_b += b
        avg_r = total_r / total_cells
        avg_g = total_g / total_cells
        avg_b = total_b / total_cells
        return f'#{int(avg_r):02x}{int(avg_g):02x}{int(avg_b):02x}'

    # Apply resonance effect to the grid
    def apply_resonance(self, frequency, root):
        self.current_frequency = frequency
        for row in self.grid:
            for cell in row:
                cell.value = (cell.value + frequency) % 10
                cell.color = cell.calculate_color()
        avg_color = self.average_color()
        root.title(f"Resonance Grid Simulation - Average Color: {avg_color}")

# Main function to set up the Tkinter window and grid


def main():
    root = tk.Tk()
    root.title("Resonance Grid Simulation")
    base_grid = pd.read_csv('./a.csv', header=None).values.tolist()

    # Replace with your method of generating grid values
    resonance_grid = ResonanceGrid(base_grid)
    # Replace with your method of generating grid values
    grid_values = [[random.randint(0, 9)
                    for _ in range(100)] for _ in range(100)]
    resonance_grid = ResonanceGrid(grid_values)

    canvas = tk.Canvas(root, width=1000, height=1000)
    canvas.pack()

    # Redraw the grid
    def redraw():
        canvas.delete("all")
        resonance_grid.display(canvas)

    # Event handler for mouse clicks
    def on_click(event):
        frequency = random.randint(1, 9)
        resonance_grid.apply_resonance(frequency, root)
        redraw()

    canvas.bind("<Button-1>", on_click)
    redraw()
    root.mainloop()


if __name__ == "__main__":
    main()


In [23]:
import pandas as pd
import random


class Cell:
    def __init__(self, value):
        self.value = value
        self.color = 'white'  # Initially colorless

    def spin_effect(self, neighbors):
        # Define the whirlwind effect here
        # For simplicity, let's say the value becomes the average of its neighbors
        self.value = sum(neighbors) / len(neighbors)

    def update_color(self):
        # Update the color based on the new value
        # Here we can define a new color mapping based on the value
        if self.value < 2:
            self.color = 'blue'
        elif self.value < 3:
            self.color = 'green'
        else:
            self.color = 'red'


class GameGrid:
    def __init__(self, matrix):
        self.grid = [[Cell(value) for value in row] for row in matrix]

    def display(self):
        for row in self.grid:
            print(' '.join(cell.color[0].upper() for cell in row))

    def spin_cells(self):
        for x in range(len(self.grid)):
            for y in range(len(self.grid[0])):
                neighbors = self.get_neighbors_values(x, y)
                self.grid[x][y].spin_effect(neighbors)
        # After spinning, update colors
        for x in range(len(self.grid)):
            for y in range(len(self.grid[0])):
                self.grid[x][y].update_color()

    def get_neighbors_values(self, x, y):
        # Get the values of neighboring cells
        neighbors = []
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < len(self.grid) and 0 <= ny < len(self.grid[0]) and (dx, dy) != (0, 0):
                    neighbors.append(self.grid[nx][ny].value)
        return neighbors


def main():
    # Initialize the game grid with a sample matrix
    base_grid = pd.read_csv('./a.csv', header=None).values.tolist()
    game_grid = GameGrid(base_grid)

    while True:
        game_grid.display()
        input("Press Enter to spin cells...")
        game_grid.spin_cells()


if __name__ == "__main__":
    main()


W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W
W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W
W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W
W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W
W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W W


B G R R R R G R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R
G R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R
R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R
R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R
R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R


In [ ]:
import tkinter as tk
import random
import colorsys
import pandas as pd

# Cell class to represent each cell in the grid


class Cell:
    def __init__(self, value, schema='default_schema', machine_set='default_set'):
        self.value = value
        self.schema = schema
        self.machine_set = machine_set
        self.color = self.calculate_color()

    # Calculate the color based on the value, schema, and machine set
    def calculate_color(self):
        hue = (self.value * 36) % 360
        if self.schema == 'schema1' and self.machine_set == 'set1':
            hue += 50
        elif self.schema == 'schema2' and self.machine_set == 'set2':
            hue += 100
        r, g, b = colorsys.hls_to_rgb(hue / 360, 0.5, 1)
        return f'#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}'

# ResonanceGrid class to manage the grid


class ResonanceGrid:
    def __init__(self, matrix):
        self.grid = [[Cell(value) for value in row] for row in matrix]
        self.current_frequency = 1

    # Display the grid on a canvas
    def display(self, canvas):
        cell_size = 10
        for x, row in enumerate(self.grid):
            for y, cell in enumerate(row):
                canvas.create_rectangle(y * cell_size, x * cell_size,
                                        (y + 1) * cell_size, (x + 1) * cell_size,
                                        fill=cell.color, outline='')

    # Calculate the average color of the grid
    def average_color(self):
        total_r = total_g = total_b = 0
        total_cells = len(self.grid) * len(self.grid[0])
        for row in self.grid:
            for cell in row:
                r, g, b = int(cell.color[1:3], 16), int(
                    cell.color[3:5], 16), int(cell.color[5:7], 16)
                total_r += r
                total_g += g
                total_b += b
        avg_r = total_r / total_cells
        avg_g = total_g / total_cells
        avg_b = total_b / total_cells
        return f'#{int(avg_r):02x}{int(avg_g):02x}{int(avg_b):02x}'

    # Apply resonance effect to the grid
    def apply_resonance(self, frequency, root):
        self.current_frequency = frequency
        for row in self.grid:
            for cell in row:
                cell.value = (cell.value + frequency) % 10
                cell.color = cell.calculate_color()
        avg_color = self.average_color()
        root.title(f"Resonance Grid Simulation - Average Color: {avg_color}")

# Main function to set up the Tkinter window and grid


def main():
    root = tk.Tk()
    root.title("Resonance Grid Simulation")
    base_grid = pd.read_csv('a.csv', header=None).values.tolist()

    # Replace with your method of generating grid values
    resonance_grid = ResonanceGrid(base_grid)

    canvas = tk.Canvas(root, width=1000, height=1000)
    canvas.pack()

    # Redraw the grid
    def redraw():
        canvas.delete("all")
        resonance_grid.display(canvas)

    # Event handler for mouse clicks
    def on_click(event):
        frequency = random.randint(1, 9)
        resonance_grid.apply_resonance(frequency, root)
        redraw()

    canvas.bind("<Button-1>", on_click)
    redraw()
    root.mainloop()


if __name__ == "__main__":
    main()


In [ ]:
import tkinter as tk
import random
import colorsys
import pandas as pd

# Use this file to keep the color of the cells constant and change the color of the window instead avg color = x = z

# Cell class to represent each cell in the grid


class Cell:
    def __init__(self, value, schema='default_schema', machine_set='default_set'):
        self.value = value
        self.schema = schema
        self.machine_set = machine_set
        self.color = self.calculate_color()

    # Calculate the color based on the value, schema, and machine set
    def calculate_color(self):
        hue = (self.value * 36) % 360
        if self.schema == 'schema1' and self.machine_set == 'set1':
            hue += 50
        elif self.schema == 'schema2' and self.machine_set == 'set2':
            hue += 100
        r, g, b = colorsys.hls_to_rgb(hue / 360, 0.5, 1)
        return f'#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}'

# ResonanceGrid class to manage the grid


class ResonanceGrid:
    def __init__(self, matrix):
        self.grid = [[Cell(value) for value in row] for row in matrix]
        self.current_frequency = 1

    # Display the grid on a canvas
    def display(self, canvas):
        cell_size = 10
        for x, row in enumerate(self.grid):
            for y, cell in enumerate(row):
                canvas.create_rectangle(y * cell_size, x * cell_size,
                                        (y + 1) * cell_size, (x + 1) * cell_size,
                                        fill=cell.color, outline='')

    # Calculate the average color of the grid
    def average_color(self):
        total_r = total_g = total_b = 0
        total_cells = len(self.grid) * len(self.grid[0])
        for row in self.grid:
            for cell in row:
                r, g, b = int(cell.color[1:3], 16), int(
                    cell.color[3:5], 16), int(cell.color[5:7], 16)
                total_r += r
                total_g += g
                total_b += b
        avg_r = total_r / total_cells
        avg_g = total_g / total_cells
        avg_b = total_b / total_cells
        return f'#{int(avg_r):02x}{int(avg_g):02x}{int(avg_b):02x}'

    # Apply resonance effect to the grid
    def apply_resonance(self, frequency, root):
        self.current_frequency = frequency
        for row in self.grid:
            for cell in row:
                cell.value = (cell.value + frequency) % 10
                cell.color = cell.calculate_color()
        avg_color = self.average_color()
        root.title(f"Resonance Grid Simulation - Average Color: {avg_color}")

# Main function to set up the Tkinter window and grid


def main():
    root = tk.Tk()
    root.title("Resonance Grid Simulation")
    base_grid = pd.read_csv('a.csv', header=None).values.tolist()

    # Replace with your method of generating grid values
    resonance_grid = ResonanceGrid(base_grid)

    canvas = tk.Canvas(root, width=1000, height=1000)
    canvas.pack()

    # Redraw the grid
    def redraw():
        canvas.delete("all")
        resonance_grid.display(canvas)

    # Event handler for mouse clicks
    def on_click(event):
        frequency = random.randint(1, 9)
        resonance_grid.apply_resonance(frequency, root)
        redraw()

    canvas.bind("<Button-1>", on_click)
    redraw()
    root.mainloop()


if __name__ == "__main__":
    main()


In [ ]:
import tkinter as tk
import random
import colorsys
import pandas as pd
import ulid

class Cell:
    def __init__(self, value, schema='default_schema', machine_set='default_set'):
        self.id = ulid.new()  # Generate a ULID for each cell
        self.value = value
        self.schema = schema
        self.machine_set = machine_set
        self.color = self.calculate_color()

    def calculate_color(self):
        # Color calculation remains the same
        hue = (self.value * 36) % 360
        if self.schema == 'schema1' and self.machine_set == 'set1':
            hue += 50
        elif self.schema == 'schema2' and self.machine_set == 'set2':
            hue += 100
        r, g, b = colorsys.hls_to_rgb(hue / 360, 0.5, 1)
        return f'#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}'

class ResonanceGrid:
    # Initialization and methods remain the same
    

    # Method to find a cell by coordinates
    def find_cell_by_coords(self, x, y, cell_size=10):
        row = y // cell_size
        col = x // cell_size
        if row < len(self.grid) and col < len(self.grid[0]):
            return self.grid[row][col]
        return None

def main():
    root = tk.Tk()
    root.title("Resonance Grid Simulation")
    base_grid = pd.read_csv('a.csv', header=None).values.tolist()

    resonance_grid = ResonanceGrid(base_grid)

    canvas = tk.Canvas(root, width=1000, height=1000)
    canvas.pack()

    def redraw():
        canvas.delete("all")
        resonance_grid.display(canvas)

    def on_click(event):
        cell = resonance_grid.find_cell_by_coords(event.x, event.y)
        if cell:
            print(f"Clicked on cell with ULID: {cell.id}")
            frequency = random.randint(1, 9)
            resonance_grid.apply_resonance(frequency, root)
            redraw()

    canvas.bind("<Button-1>", on_click)
    redraw()
    root.mainloop()

if __name__ == "__main__":
    main()


In [ ]:
import tkinter as tk
import random
import colorsys
import pandas as pd

class Ulid: # A simple ULID generator
    def __init__(self):
        self.counter = 0

    def new(self):
        self.counter += 1
        return f"00000000-0000-0000-0000-{self.counter:012x}"
    
ulid = Ulid()

class Cell:
    def __init__(self, value, schema='default_schema', machine_set='default_set'):
        self.id = ulid.new()  # Generate a ULID for each cell
        self.value = value
        self.schema = schema
        self.machine_set = machine_set
        self.color = self.calculate_color()

    def calculate_color(self):
        hue = (self.value * 36) % 360
        if self.schema == 'schema1' and self.machine_set == 'set1':
            hue += 50
        elif self.schema == 'schema2' and self.machine_set == 'set2':
            hue += 100
        r, g, b = colorsys.hls_to_rgb(hue / 360, 0.5, 1)
        return f'#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}'

class ResonanceGrid:
    def __init__(self, matrix):
        self.grid = [[Cell(value) for value in row] for row in matrix]
        self.current_frequency = 1

    def display(self, canvas):
        cell_size = 10
        for x, row in enumerate(self.grid):
            for y, cell in enumerate(row):
                canvas.create_rectangle(y * cell_size, x * cell_size,
                                        (y + 1) * cell_size, (x + 1) * cell_size,
                                        fill=cell.color, outline='')

    def average_color(self):
        total_r = total_g = total_b = 0
        total_cells = sum(len(row) for row in self.grid)
        for row in self.grid:
            for cell in row:
                r, g, b = int(cell.color[1:3], 16), int(cell.color[3:5], 16), int(cell.color[5:7], 16)
                total_r += r
                total_g += g
                total_b += b
        avg_r = total_r / total_cells
        avg_g = total_g / total_cells
        avg_b = total_b / total_cells
        return f'#{int(avg_r):02x}{int(avg_g):02x}{int(avg_b):02x}'

    def apply_resonance(self, frequency, root):
        self.current_frequency = frequency
        for row in self.grid:
            for cell in row:
                cell.value = (cell.value + frequency) % 10
                cell.color = cell.calculate_color()
        avg_color = self.average_color()
        root.title(f"Resonance Grid Simulation - Average Color: {avg_color}")

    def find_cell_by_coords(self, x, y, cell_size=10):
        row = y // cell_size
        col = x // cell_size
        if row < len(self.grid) and col < len(self.grid[0]):
            return self.grid[row][col]
        return None

def redraw(canvas, resonance_grid):
    canvas.delete("all")
    resonance_grid.display(canvas)

def main():
    root = tk.Tk()
    root.geometry("1000x1000")
    root.title("Resonance Grid Simulation")
    base_grid = pd.read_csv('a.csv', header=None).values.tolist()

    resonance_grid = ResonanceGrid(base_grid)
    canvas = tk.Canvas(root, width=1000, height=1000)
    canvas.pack()

    def on_click(event):
        cell = resonance_grid.find_cell_by_coords(event.x, event.y)
        if cell:
            print(f"Clicked on cell with ULID: {cell.id}")
            frequency = random.randint(1, 9)
            resonance_grid.apply_resonance(frequency, root)
            redraw(canvas, resonance_grid)

    canvas.bind("<Button-1>", on_click)
    redraw(canvas, resonance_grid)

    root.mainloop()

if __name__ == "__main__":
    main()


In [ ]:
c

In [ ]:
import tkinter as tk
import random
import colorsys
import pandas as pd

class Ulid:  # A simple ULID generator
    def __init__(self):
        self.counter = 0

    def new(self):
        self.counter += 1
        return f"00000000-0000-0000-0000-{self.counter:012x}"

ulid = Ulid()

class CellObserver:
    def update_cell(self, cell_id, new_value):
        pass

class MockServer(CellObserver):
    def __init__(self):
        self.cells = {}  # Store cell states or other relevant information

    def update_cell(self, cell_id, new_value):
        # Process cell update, e.g., log, change state, or decide on a new value
        print(f"Cell {cell_id} updated with new value: {new_value}")
        # Here, you could simulate decision-making and update a subset of cells
        # This part is simplified for demonstration

class Cell:
    def __init__(self, value, observer=None, schema='default_schema', machine_set='default_set'):
        self.id = ulid.new()
        self.value = value
        self.schema = schema
        self.machine_set = machine_set
        self.color = self.calculate_color()
        self.observer = observer  # Observer reference

    def calculate_color(self):
        hue = (self.value * 36) % 360
        if self.schema == 'schema1' and self.machine_set == 'set1':
            hue += 50
        elif self.schema == 'schema2' and self.machine_set == 'set2':
            hue += 100
        r, g, b = colorsys.hls_to_rgb(hue / 360, 0.5, 1)
        return f'#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}'

    def set_value(self, new_value):
        self.value = new_value
        self.color = self.calculate_color()
        # Notify observer about the change
        if self.observer:
            self.observer.update_cell(self.id, new_value)

class ResonanceGrid:
    def __init__(self, matrix, observer=None):
        self.grid = [[Cell(value, observer=observer) for value in row] for row in matrix]
        self.current_frequency = 1

    def display(self, canvas):
        cell_size = 10
        for x, row in enumerate(self.grid):
            for y, cell in enumerate(row):
                canvas.create_rectangle(y * cell_size, x * cell_size,
                                        (y + 1) * cell_size, (x + 1) * cell_size,
                                        fill=cell.color, outline='')

    def apply_resonance(self, frequency, root):
        self.current_frequency = frequency
        for row in self.grid:
            for cell in row:
                new_value = (cell.value + frequency) % 10
                cell.set_value(new_value)
        # Optionally, update the title or other UI elements to reflect changes

def redraw(canvas, resonance_grid):
    canvas.delete("all")
    resonance_grid.display(canvas)

def main():
    root = tk.Tk()
    root.geometry("1000x1000")
    root.title("Resonance Grid Simulation")
    
    # Example data loading; replace 'a.csv' with your actual CSV file path
    try:
        base_grid = pd.read_csv('a.csv', header=None).values.tolist()
    except FileNotFoundError:
        base_grid = [[random.randint(0, 9) for _ in range(100)] for _ in range(100)]  # Fallback dummy data

    mock_server = MockServer()
    resonance_grid = ResonanceGrid(base_grid, observer=mock_server)
    canvas = tk.Canvas(root, width=1000, height=1000)
    canvas.pack()

    def on_click(event):
        # This is where you could simulate selecting and updating a subset of cells
        frequency = random.randint(1, 9)
        resonance_grid.apply_resonance(frequency, root)
        redraw(canvas, resonance_grid)

    canvas.bind("<Button-1>", on_click)
    redraw(canvas, resonance_grid)

    root.mainloop()

if __name__ == "__main__":
    main()


In [ ]:
import tkinter as tk
import random
import colorsys
import pandas as pd
from tkinter import Toplevel

class Ulid:  # A simple ULID generator
    def __init__(self):
        self.counter = 0

    def new(self):
        self.counter += 1
        return f"00000000-0000-0000-0000-{self.counter:012x}"

ulid = Ulid()

class Cell:
    def __init__(self, value, observer=None):
        self.id = ulid.new()
        self.value = value
        self.color = self.calculate_color()

    def calculate_color(self):
        hue = (self.value * 36) % 360
        r, g, b = colorsys.hls_to_rgb(hue / 360, 0.5, 1)
        return f'#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}'

    def set_value(self, new_value):
        self.value = new_value
        self.color = self.calculate_color()

class GridManager:
    def __init__(self, matrix):
        self.grid = [[Cell(value) for value in row] for row in matrix]

    def apply_resonance(self, frequency):
        for row in self.grid:
            for cell in row:
                new_value = (cell.value + frequency) % 10
                cell.set_value(new_value)

    def get_grid_state(self):
        flat_grid = [cell for row in self.grid for cell in row]
        color_codes = {cell.id: cell.color for cell in flat_grid}
        average_color = self.calculate_average_color(flat_grid)
        return color_codes, average_color

    def calculate_average_color(self, cells):
        num_cells = len(cells)
        if num_cells == 0:
            return '#000000'
        total_r = total_g = total_b = 0
        for cell in cells:
            hex_color = cell.color.lstrip('#')
            r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
            total_r += r
            total_g += g
            total_b += b
        avg_r = int(total_r / num_cells)
        avg_g = int(total_g / num_cells)
        avg_b = int(total_b / num_cells)
        return f'#{avg_r:02x}{avg_g:02x}{avg_b:02x}'

# Example usage
def initialize_grid():
    try:
        base_grid = pd.read_csv('a.csv', header=None).values.tolist()
    except FileNotFoundError:
        base_grid = [[random.randint(0, 9) for _ in range(10)] for _ in range(10)]  # Adjusted to 10x10 for simplicity

    grid_manager = GridManager(base_grid)
    return grid_manager

# This example just initializes the grid and returns its state.
# In a real application, you might call grid_manager.apply_resonance(frequency) to simulate updates
grid_manager = initialize_grid()
color_codes, average_color = grid_manager.get_grid_state()
print(color_codes, average_color)

## Return the color codes and average color to the main function to display the grid and average color



In [ ]:
def encode_decode(message, sequence, encode=True):
    """
    Encodes or decodes a message using a provided sequence.
    
    :param message: The message to encode or decode.
    :param sequence: A list of integers used for the encoding/decoding process.
    :param encode: Determines whether to encode (True) or decode (False) the message.
    :return: The encoded or decoded message.
    """
    transformed_message = []
    sequence_len = len(sequence)
    
    for i, char in enumerate(message):
        sequence_index = i % sequence_len
        shift = sequence[sequence_index]
        
        if not encode:
            shift = -shift
        
        # Shift character's ASCII value and wrap around if necessary
        transformed_char = chr((ord(char) + shift - 32) % 95 + 32)
        transformed_message.append(transformed_char)
    
    return ''.join(transformed_message)

# Example usage
base_sequence = [1,1,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11] # Truncated for brevity
message = "Hello, World!"
encoded_message = encode_decode(message, base_sequence, encode=True)
decoded_message = encode_decode(encoded_message, base_sequence, encode=False)

print(f"Original: {message}")
print(f"Encoded: {encoded_message}")
print(f"Decoded: {decoded_message}")

// fibonacci we can represent infinity 

// because our grid uses a special pattern + fibonacci modlo /10 

we can always represent a subset and sync with 'infinity' as a whole consistently without worrying about overwrites because infinity started at the begining of time apex domain 

the modlo and grid outcome + a index to the time synced (infinity) will always be unique and consistent 

we just divide them out 

x = y 
r^2 = 1 

actor.apex.com/ulid/1


indexulid | map[string][n] 

n = {....r}


In [ ]:
def encode_decode(message, sequence, encode=True):
    """
    Encodes or decodes a message using a provided sequence.
    
    :param message: The message to encode or decode.
    :param sequence: A list of integers used for the encoding/decoding process.
    :param encode: Determines whether to encode (True) or decode (False) the message.
    :return: The encoded or decoded message.
    """
    transformed_message = []
    sequence_len = len(sequence)
    
    for i, char in enumerate(message):
        sequence_index = i % sequence_len
        shift = sequence[sequence_index]
        
        if not encode:
            shift = -shift
        
        # Shift character's ASCII value and wrap around if necessary
        transformed_char = chr((ord(char) + shift - 32) % 95 + 32)
        transformed_message.append(transformed_char)
    
    return ''.join(transformed_message)

# Example usage
base_sequence = [1,1,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11] # Truncated for brevity
message = "Hello, World!"
encoded_message = encode_decode(message, base_sequence, encode=True)
decoded_message = encode_decode(encoded_message, base_sequence, encode=False)

print(f"Original: {message}")
print(f"Encoded: {encoded_message}")
print(f"Decoded: {decoded_message}")


In [ ]:
[
        1, 1, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 2, 2, 4, 6, 10, 1, 1, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 3, 3, 6, 9, 15, 6, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 4, 8, 12, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 5, 5, 10, 1, 1, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 6, 6, 12, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 7, 7, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 8, 8,
        16, 7, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 9, 9, 18, 9, 17, 8, 15, 6, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        10, 1, 1, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 12, 3, 5, 8, 13, 13, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5,
        9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 15, 6,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 16, 7, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        17, 8, 15, 6, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 18, 9, 17, 8, 15, 6, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 19, 10, 1, 1, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 20, 2, 2, 4, 6, 10, 1, 1,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 21, 3, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 22, 4, 6, 10, 1, 1,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 23, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 24, 6, 10, 1,
        1, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 25, 7, 12, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 26, 8,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 27, 9, 16, 7, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 28, 10, 1, 1, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 29, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 30, 3, 3, 6, 9, 15, 6, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 31, 4, 5,
        9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 32, 5, 7, 12, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 33, 6, 9, 15, 6, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 34, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 35, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 36, 9, 15, 6, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 37, 10, 1, 1, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 38, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 39, 12, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 40, 4,
        4, 8, 12, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 41, 5, 6, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        42, 6, 8, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 43, 7, 10, 1, 1,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 44, 8, 12, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 45, 9, 14,
        5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5,
        9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 46, 10, 1, 1, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 47, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 48, 12, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 49, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 50, 5, 5, 10, 1, 1, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 51, 6, 7, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 52, 7, 9, 16, 7,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 53, 8, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 54, 9, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 55, 10, 1, 1, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 56,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 57, 12, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 58, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 59, 14, 5, 9, 14, 5, 9, 14, 5,
        9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 60, 6, 6, 12, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 61, 7, 8, 15,
        6, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 11, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 5, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 62, 8, 10, 1, 1, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 63, 9, 12,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 64, 10, 1, 1, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 65,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 66, 12, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 67, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 68, 14, 5, 9, 14, 5, 9, 14, 5,
        9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 69, 15, 6, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 70, 7, 7, 14,
        5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5,
        9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 71, 8, 9, 17, 8, 15, 6, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 72, 9, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 73, 10, 1, 1, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 74, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 75, 12, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 76, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 77, 14, 5, 9, 14,
        5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5,
        9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 78, 15, 6, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        79, 16, 7, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 80, 8, 8, 16, 7, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 81, 9, 10, 1, 1, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 82, 10, 1, 1, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 83, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 84, 12, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 85, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 86, 14, 5,
        9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 87, 15, 6, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 88, 16, 7, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 89, 17, 8, 15, 6, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 90, 9, 9, 18, 9, 17, 8, 15, 6, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 91, 10, 1, 1,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 92, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 93, 12,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 94, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        95, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5,
        9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9,
        14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 14, 5, 9, 96, 15, 6, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 97, 16, 7, 13, 2, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 98, 17, 8,
        15, 6, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5,
        8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2,
        3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7,
        11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13,
        4, 7, 11, 2, 3, 5, 8, 13, 4, 99, 18, 9, 17, 6, 15, 6, 11, 2, 3, 5, 8,
        13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3,
        5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11,
        2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4,
        7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8, 13, 4, 7, 11, 2, 3, 5, 8,
        100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0,
      ];

In [ ]:
import numpy as np

class DataPrism:
    def __init__(self, base_size, layers):
        """
        Initialize the DataPrism with a specified base size and number of layers.
        
        :param base_size: The size of the 1D base array.
        :param layers: The number of layers above the base.
        """
        self.base = np.zeros(base_size, dtype=int) # 1D base array
        self.layers = [np.zeros((base_size, base_size)) for _ in range(layers)] # Initialize layers

    def fill_base(self, data):
        """
        Fill the base layer with data.
        
        :param data: A 1D array of integers to fill the base.
        """
        if len(data) == len(self.base):
            self.base = data
        else:
            raise ValueError("Data length must match base size.")

    def update_layer(self, layer_index, data):
        """
        Update a specific layer with new data.
        
        :param layer_index: The index of the layer to update.
        :param data: A 2D array of data to insert into the layer.
        """
        if 0 <= layer_index < len(self.layers):
            if data.shape == self.layers[layer_index].shape:
                self.layers[layer_index] = data
            else:
                raise ValueError("Data shape must match layer shape.")
        else:
            raise IndexError("Layer index out of bounds.")

    def get_layer(self, layer_index):
        """
        Retrieve data from a specific layer.
        
        :param layer_index: The index of the layer to retrieve.
        :return: The data stored in the specified layer.
        """
        if 0 <= layer_index < len(self.layers):
            return self.layers[layer_index]
        else:
            raise IndexError("Layer index out of bounds.")

# Initialize the DataPrism with a base size of 100 and 5 layers
prism = DataPrism(100, 5)

# Fill the base with integers
base_data = np.arange(100) # Example: integers 0 to 99
prism.fill_base(base_data)

# Update the first layer with some data
layer_data = np.random.rand(100, 100) # Example: random data for the layer
prism.update_layer(0, layer_data)

# Retrieve data from the first layer
retrieved_data = prism.get_layer(0)

# Outputs
base_data, retrieved_data[:5, :5]  # Show a portion of the data for brevity


In [ ]:
import numpy as np

class DataPrism:
    def __init__(self, base_size, layers):
        """
        Initialize the DataPrism with a specified base size and number of layers.
        
        :param base_size: The size of the 1D base array.
        :param layers: The number of layers above the base.
        """
        self.base = np.zeros(base_size, dtype=int) # 1D base array
        self.layers = [np.zeros((base_size, base_size)) for _ in range(layers)] # Initialize layers

    def fill_base(self, data):
        """
        Fill the base layer with data.
        
        :param data: A 1D array of integers to fill the base.
        """
        if len(data) == len(self.base):
            self.base = data
        else:
            raise ValueError("Data length must match base size.")

    def update_layer(self, layer_index, data):
        """
        Update a specific layer with new data.
        
        :param layer_index: The index of the layer to update.
        :param data: A 2D array of data to insert into the layer.
        """
        if 0 <= layer_index < len(self.layers):
            if data.shape == self.layers[layer_index].shape:
                self.layers[layer_index] = data
            else:
                raise ValueError("Data shape must match layer shape.")
        else:
            raise IndexError("Layer index out of bounds.")

    def get_layer(self, layer_index):
        """
        Retrieve data from a specific layer.
        
        :param layer_index: The index of the layer to retrieve.
        :return: The data stored in the specified layer.
        """
        if 0 <= layer_index < len(self.layers):
            return self.layers[layer_index]
        else:
            raise IndexError("Layer index out of bounds.")

# Initialize the DataPrism with a base size of 100 and 5 layers
prism = DataPrism(100, 5)

# Fill the base with integers
base_data = np.arange(100) # Example: integers 0 to 99
prism.fill_base(base_data)

# Update the first layer with some data
layer_data = np.random.rand(100, 100) # Example: random data for the layer
prism.update_layer(0, layer_data)

# Retrieve data from the first layer
retrieved_data = prism.get_layer(0)

# Outputs
base_data, retrieved_data[:5, :5]  # Show a portion of the data for brevity


In [ ]:
import colorsys
import random

import pandas as pd
from flask import Flask, jsonify, request
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

class Ulid:  # A simple ULID generator
    def __init__(self):
        self.counter = 0

    def new(self):
        self.counter += 1
        return f"00000000-0000-0000-0000-{self.counter:012x}"

ulid = Ulid()

class Cell:
    def __init__(self, value):
        self.id = ulid.new()
        self.value = value
        self.color = self.calculate_color()

    def calculate_color(self):
        hue = (self.value * 36) % 360
        r, g, b = colorsys.hls_to_rgb(hue / 360, 0.5, 1)
        return f'#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}'

    def set_value(self, new_value):
        self.value = new_value
        self.color = self.calculate_color()

class GridManager:
    def __init__(self, matrix):
        self.grid = [[Cell(value) for value in row] for row in matrix]

    def apply_resonance(self, frequency):
        for row in self.grid:
            for cell in row:
                new_value = (cell.value + frequency) % 10
                cell.set_value(new_value)

    def get_grid_state(self):
        flat_grid = [cell for row in self.grid for cell in row]
        color_codes = {cell.id: cell.color for cell in flat_grid}
        average_color = self.calculate_average_color(flat_grid)
        return color_codes, average_color

    def calculate_average_color(self, cells):
        num_cells = len(cells)
        if num_cells == 0:
            return '#000000'
        total_r = total_g = total_b = 0
        for cell in cells:
            hex_color = cell.color.lstrip('#')
            r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
            total_r += r
            total_g += g
            total_b += b
        avg_r = int(total_r / num_cells)
        avg_g = int(total_g / num_cells)
        avg_b = int(total_b / num_cells)
        return f'#{avg_r:02x}{avg_g:02x}{avg_b:02x}'

# Initialize grid manager with dummy data
try:
    base_grid = pd.read_csv('a.csv', header=None).values.tolist()
except FileNotFoundError:
    base_grid = [[random.randint(0, 9) for _ in range(10)] for _ in range(10)]  # Adjusted to 10x10 for simplicity

grid_manager = GridManager(base_grid)

@app.route('/click', methods=['POST'])
def handle_click():
    data = request.json
    print(data)
    frequency = data.get('frequency', 1)  # Default frequency to 1 if not provided
    grid_manager.apply_resonance(frequency)
    return jsonify(success=True)

@app.route('/grid', methods=['GET'])
def get_grid():
    color_codes, average_color = grid_manager.get_grid_state()
    return jsonify({
        'color_codes': color_codes,
        'average_color': average_color
    })
    
@app.route('/', methods=['GET'])
def index():
    # generate the grid and return the color codes and average color
    color_codes, average_color = grid_manager.get_grid_state()
    return jsonify({
        'color_codes': color_codes,
        'average_color': average_color
    })
    
    
    

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
import numpy as np
import pandas as pd 

class DataPrism:
    def __init__(self, base_size, layers):
        """
        Initialize the DataPrism with a specified base size and number of layers.
        
        :param base_size: The size of the 1D base array.
        :param layers: The number of layers above the base.
        """
        self.base = np.zeros(base_size, dtype=int) # 1D base array
        self.layers = [np.zeros((base_size, base_size)) for _ in range(layers)] # Initialize layers

    def fill_base(self, data):
        """
        Fill the base layer with data.
        
        :param data: A 1D array of integers to fill the base.
        """
        if len(data) == len(self.base):
            self.base = data
        else:
            raise ValueError("Data length must match base size.")

    def update_layer(self, layer_index, data):
        """
        Update a specific layer with new data.
        
        :param layer_index: The index of the layer to update.
        :param data: A 2D array of data to insert into the layer.
        """
        if 0 <= layer_index < len(self.layers):
            if data.shape == self.layers[layer_index].shape:
                self.layers[layer_index] = data
            else:
                raise ValueError("Data shape must match layer shape.")
        else:
            raise IndexError("Layer index out of bounds.")

    def get_layer(self, layer_index):
        """
        Retrieve data from a specific layer.
        
        :param layer_index: The index of the layer to retrieve.
        :return: The data stored in the specified layer.
        """
        if 0 <= layer_index < len(self.layers):
            return self.layers[layer_index]
        else:
            raise IndexError("Layer index out of bounds.")

# Initialize the DataPrism with a base size of 100 and 5 layers
prism = DataPrism(100, 5)

# Fill the base with integers
base_data = pd.read_csv('a.csv', header=None).values.tolist()
prism.fill_base(base_data)

# Update the first layer with some data
layer_data = np.random.rand(100, 100) # Example: random data for the layer
prism.update_layer(0, layer_data)

# Retrieve data from the first layer
retrieved_data = prism.get_layer(0)

# Outputs
base_data, retrieved_data[:100, :100]  # Show a portion of the data for brevity


In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt


class DataPrism:
    def __init__(self, base_size, layers):
        """
        Initialize the DataPrism with a specified base size and number of layers.
        
        :param base_size: The size of the 1D base array.
        :param layers: The number of layers above the base.
        """
        self.base = np.zeros(base_size, dtype=int) # 1D base array
        self.layers = [np.zeros((base_size, base_size)) for _ in range(layers)] # Initialize layers

    def fill_base(self, data):
        """
        Fill the base layer with data.
        
        :param data: A 1D array of integers to fill the base.
        """
        if len(data) == len(self.base):
            self.base = data
        else:
            raise ValueError("Data length must match base size.")

    def update_layer(self, layer_index, data):
        """
        Update a specific layer with new data.
        
        :param layer_index: The index of the layer to update.
        :param data: A 2D array of data to insert into the layer.
        """
        if 0 <= layer_index < len(self.layers):
            if data.shape == self.layers[layer_index].shape:
                self.layers[layer_index] = data
            else:
                raise ValueError("Data shape must match layer shape.")
        else:
            raise IndexError("Layer index out of bounds.")

    def get_layer(self, layer_index):
        """
        Retrieve data from a specific layer.
        
        :param layer_index: The index of the layer to retrieve.
        :return: The data stored in the specified layer.
        """
        if 0 <= layer_index < len(self.layers):
            return self.layers[layer_index]
        else:
            raise IndexError("Layer index out of bounds.")
        
    def calculate_mean_across_layers(self):
        """Calculate the mean value at each position across all layers."""
        # Stack all layers to create a 3D array and then calculate the mean along the third dimension
        stacked_layers = np.stack(self.layers, axis=2)
        return np.mean(stacked_layers, axis=2)

# Initialize the DataPrism with a base size of 100 and 5 layers
prism = DataPrism(100, 5)

# Assuming the desired base size is 100
desired_base_size = 100

# Assuming an example base_data array
original_data  = [1,1,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11,2,3,5,8,13,4,7,11] # Truncated for brevity
 # Simulate loading data from a file
# Adjusting the dataset
if len(original_data) > desired_base_size:
    adjusted_data = original_data[:desired_base_size]
elif len(original_data) < desired_base_size:
    adjusted_data = original_data + [0] * (desired_base_size - len(original_data))  # Padding with zeros
else:
    adjusted_data = original_data

# Now use adjusted_data to fill the base
prism.fill_base(adjusted_data)
# Update the first layer with some data
layer_data = np.random.rand(100, 100) # Example: random data for the layer
prism.update_layer(0, layer_data)

# Retrieve data from the first layer
retrieved_data = prism.get_layer(0)

# Outputs
# base_data, retrieved_data[:2, :10]  # Show a portion of the data for brevity


# Assuming you have a DataPrism instance named `prism`
layer_index = 0  # Choose which layer to visualize
layer_data = prism.get_layer(layer_index)
layer_data = pd.read_csv('a.csv', header=None).values.tolist()

# Visualize the layer
plt.figure(figsize=(10, 10))
plt.imshow(layer_data, cmap='viridis', interpolation='nearest')
plt.title(f'Layer {layer_index} Visualization')
plt.colorbar(label='Data Value')
plt.show()

# Add a value random layer of 100x100 cells linked between the cells in base layer 0 than to cells in same position in layer 1 and so on
# as if the cells in the base layer are the source of the data and the cells in the layers are the destination of the data
def add_random_layer(prism, layer_index):
    """Add a random layer of 100x100 cells."""
    random_layer = np.random.rand(100, 100)
    prism.update_layer(layer_index, random_layer)
    
# Add a random layer to the DataPrism
add_random_layer(prism, 1)

# Visualize the new layer
layer_index = 1  # Choose which layer to visualize
layer_data = prism.get_layer(layer_index)

# Visualize the layer
plt.figure(figsize=(10, 10))
plt.imshow(layer_data, cmap='viridis', interpolation='nearest')

plt.title(f'Layer {layer_index} Visualization')
plt.colorbar(label='Data Value')
plt.show()

# Vidualize the 3 layers stacked but link between the layers as it was mapped like a base_layer sandwich with random layers in between
mean_values = prism.calculate_mean_across_layers()
plt.figure(figsize=(10, 10))
plt.imshow(mean_values, cmap='viridis', interpolation='nearest')
plt.title('Mean Values Across Layers')
plt.colorbar(label='Mean Data Value')
plt.show()



# plt.figure(figsize=(10, 10))
# plt.imshow(layer_data, cmap='viridis', interpolation='nearest')
# plt.title(f'Layer {layer_index} Visualization')
# plt.colorbar(label='Data Value')
# plt.show()


# mean_values = prism.calculate_mean_across_layers()
# plt.figure(figsize=(10, 10))
# plt.imshow(mean_values, cmap='viridis', interpolation='nearest')
# plt.title('Mean Values Across Layers')
# plt.colorbar(label='Mean Data Value')
# plt.show()

# # Example: Mapping labels to specific positions within the layer
# label_mapping = {
#     'Region A': (20, 30),  # Example position for Region A
#     'Region B': (50, 60),  # Example position for Region B
#     # Add more mappings as needed
# }

# # Visualize the layer with labels
# plt.figure(figsize=(10, 10))
# plt.imshow(layer_data, cmap='viridis', interpolation='nearest')
# for label, position in label_mapping.items():
#     plt.text(position[1], position[0], label, color='red', fontsize=12)
# plt.colorbar(label='Data Value')
# plt.show()





In [19]:
import colorsys
import random
import tkinter as tk

import pandas as pd

# Cell class to represent each cell in the grid
class Cell:
    def __init__(self, value, schema='default_schema', machine_set='default_set'):
        self.value = value
        self.schema = schema
        self.machine_set = machine_set
        self.color = self.calculate_color()

    # Calculate the color based on the value, schema, and machine set
    def calculate_color(self):
        hue = (self.value * 36) % 360
        if self.schema == 'schema1' and self.machine_set == 'set1':
            hue += 50
        elif self.schema == 'schema2' and self.machine_set == 'set2':
            hue += 100
        r, g, b = colorsys.hls_to_rgb(hue / 360, 0.5, 1)
        return f'#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}'

# ResonanceGrid class to manage the grid
class ResonanceGrid:
    def __init__(self, matrix):
        self.grid = [[Cell(value) for value in row] for row in matrix]
        self.current_frequency = 1

    # Display the grid on a canvas
    def display(self, canvas):
        cell_size = 10
        for x, row in enumerate(self.grid):
            for y, cell in enumerate(row):
                canvas.create_rectangle(y * cell_size, x * cell_size,
                                        (y + 1) * cell_size, (x + 1) * cell_size,
                                        fill=cell.color, outline='')

    # Calculate the average color of the grid
    def average_color(self):
        total_r = total_g = total_b = 0
        total_cells = len(self.grid) * len(self.grid[0])
        for row in self.grid:
            for cell in row:
                r, g, b = int(cell.color[1:3], 16), int(
                    cell.color[3:5], 16), int(cell.color[5:7], 16)
                total_r += r
                total_g += g
                total_b += b
        avg_r = total_r / total_cells
        avg_g = total_g / total_cells
        avg_b = total_b / total_cells
        return f'#{int(avg_r):02x}{int(avg_g):02x}{int(avg_b):02x}'

    # Apply resonance effect to the grid
    def apply_resonance(self, frequency, root):
        self.current_frequency = frequency
        for row in self.grid:
            for cell in row:
                cell.value = (cell.value + frequency) % 10
                cell.color = cell.calculate_color()
        avg_color = self.average_color()
        root.title(f"Resonance Grid Simulation - Average Color: {avg_color}")

# Main function to set up the Tkinter window and grid
def main():
    root = tk.Tk()
    root.title("Resonance Grid Simulation")
    base_grid = pd.read_csv('a.csv', header=None).values.tolist()

    # Replace with your method of generating grid values
    resonance_grid = ResonanceGrid(base_grid)

    canvas = tk.Canvas(root, width=1000, height=1000)
    canvas.pack()

    # Redraw the grid
    def redraw():
        canvas.delete("all")
        resonance_grid.display(canvas)

    # Event handler for mouse clicks
    def on_click(event):
        frequency = random.randint(1, 9)
        resonance_grid.apply_resonance(frequency, root)
        redraw()

    canvas.bind("<Button-1>", on_click)
    redraw()
    root.mainloop()


if __name__ == "__main__":
    main()


In [6]:
import tkinter as tk
import pandas as pd
import colorsys
import random

class Cell:
    def __init__(self, value, schema='default_schema', machine_set='default_set'):
        self.value = value
        self.schema = schema
        self.machine_set = machine_set
        self.color = self.calculate_color()
        self.marked = 0  # Changed to count the number of clicks

    def calculate_color(self):
        # Simplified color calculation for demonstration
        hue = (self.value * 36) % 360
        r, g, b = colorsys.hls_to_rgb(hue / 360, 0.5, 1)
        return f'#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}'

class ResonanceGrid:
    def __init__(self, matrix):
        self.grid = [[Cell(value) for value in row] for row in matrix]
        self.cell_size = 20  # Increased cell size for easier clicking

    def display_grid(self, canvas):
        for x, row in enumerate(self.grid):
            for y, cell in enumerate(row):
                x0, y0 = y * self.cell_size, x * self.cell_size
                x1, y1 = x0 + self.cell_size, y0 + self.cell_size
                canvas.create_rectangle(x0, y0, x1, y1, fill=cell.color, outline='')

    def mark_cell(self, x, y):
        grid_x, grid_y = y // self.cell_size, x // self.cell_size
        if 0 <= grid_x < len(self.grid) and 0 <= grid_y < len(self.grid[0]):
            self.grid[grid_x][grid_y].marked += 1  # Increment click count

    def display_clicks(self, canvas, width, height):
        max_clicks = max(cell.marked for row in self.grid for cell in row) if self.grid else 0
        bar_width = width / len(self.grid[0])
        for x, row in enumerate(self.grid):
            for y, cell in enumerate(row):
                if cell.marked > 0:
                    bar_height = (cell.marked / max_clicks) * height
                    x0, y0 = y * bar_width, height - bar_height
                    x1, y1 = x0 + bar_width, height
                    canvas.create_rectangle(x0, y0, x1, y1, fill='blue', outline='')

def main():
    root = tk.Tk()
    root.title("Resonance Grid Simulation")

    # Example grid generation (replace with CSV loading if needed)
    base_grid = pd.read_csv('a.csv', header=None).values.tolist()
    
    resonance_grid = ResonanceGrid(base_grid)

    grid_canvas = tk.Canvas(root, width=400, height=400)
    grid_canvas.pack(side=tk.LEFT)

    click_canvas = tk.Canvas(root, width=400, height=400, bg='white')
    click_canvas.pack(side=tk.RIGHT)

    def redraw_click_canvas():
        click_canvas.delete("all")
        resonance_grid.display_clicks(click_canvas, 400, 400)

    def on_click(event):
        resonance_grid.mark_cell(event.x, event.y)
        grid_canvas.delete("all")
        resonance_grid.display_grid(grid_canvas)
        redraw_click_canvas()

    grid_canvas.bind("<Button-1>", on_click)
    resonance_grid.display_grid(grid_canvas)
    redraw_click_canvas()
    root.mainloop()

if __name__ == "__main__":
    main()


In [8]:
import tkinter as tk
import pandas as pd
import colorsys
import random

class Cell:
    def __init__(self, value, schema='default_schema', machine_set='default_set'):
        self.value = value
        self.schema = schema
        self.machine_set = machine_set
        self.color = self.calculate_color()
        self.marked = 0  # Changed to count the number of clicks

    def calculate_color(self):
        # Simplified color calculation for demonstration
        hue = (self.value * 36) % 360
        r, g, b = colorsys.hls_to_rgb(hue / 360, 0.5, 1)
        return f'#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}'

class ResonanceGrid:
    def __init__(self, matrix):
        self.grid = [[Cell(value) for value in row] for row in matrix]
        self.cell_size = 20  # Increased cell size for easier clicking

    def display_grid(self, canvas):
        for x, row in enumerate(self.grid):
            for y, cell in enumerate(row):
                x0, y0 = y * self.cell_size, x * self.cell_size
                x1, y1 = x0 + self.cell_size, y0 + self.cell_size
                canvas.create_rectangle(x0, y0, x1, y1, fill=cell.color, outline='')

    def mark_cell(self, x, y):
        grid_x, grid_y = y // self.cell_size, x // self.cell_size
        if 0 <= grid_x < len(self.grid) and 0 <= grid_y < len(self.grid[0]):
            self.grid[grid_x][grid_y].marked += 1  # Increment click count

    def display_clicks(self, canvas, width, height):
        max_clicks = max(cell.marked for row in self.grid for cell in row) if self.grid else 0
        bar_width = width / len(self.grid[0])
        for x, row in enumerate(self.grid):
            for y, cell in enumerate(row):
                if cell.marked > 0:
                    bar_height = (cell.marked / max_clicks) * height
                    x0, y0 = y * bar_width, height - bar_height
                    x1, y1 = x0 + bar_width, height
                    canvas.create_rectangle(x0, y0, x1, y1, fill='blue', outline='')

def main():
    root = tk.Tk()
    root.title("Resonance Grid Simulation")

    # Example grid generation (replace with CSV loading if needed)
    base_grid = pd.read_csv('a.csv', header=None).values.tolist()
    
    resonance_grid = ResonanceGrid(base_grid)

    grid_canvas = tk.Canvas(root, width=400, height=400)
    grid_canvas.pack(side=tk.LEFT)

    click_canvas = tk.Canvas(root, width=400, height=400, bg='white')
    click_canvas.pack(side=tk.RIGHT)

    def redraw_click_canvas():
        click_canvas.delete("all")
        resonance_grid.display_clicks(click_canvas, 400, 400)

    def on_click(event):
        resonance_grid.mark_cell(event.x, event.y)
        grid_canvas.delete("all")
        resonance_grid.display_grid(grid_canvas)
        redraw_click_canvas()

    grid_canvas.bind("<Button-1>", on_click)
    resonance_grid.display_grid(grid_canvas)
    redraw_click_canvas()
    root.mainloop()

if __name__ == "__main__":
    main()
